# Modelo NLP - Twitter

In [ ]:
# Installs
#!pip3 install scikit-learn
#!pip3 install -U spacy
#!python3 -m spacy download es
#!python3 -m spacy download es_core_news_sm

In [ ]:
#!pip install spacy

In [ ]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download es_core_news_sm

In [16]:
import pandas as pd
from spacy import displacy
import spacy
import re

In [2]:
from spacy.lang.es.stop_words import STOP_WORDS

In [3]:
nlp = spacy.load('es_core_news_sm')

In [4]:
text = 'Hay un gato en la casa. Hay una granja en la nevera. Esta es una tercera frase'


In [5]:
doc = nlp (text)

In [6]:
for token in doc: #ver los tokens/elemento de texto que encuentra la libreria
    print(token)

Hay
un
gato
en
la
casa
.
Hay
una
granja
en
la
nevera
.
Esta
es
una
tercera
frase


In [7]:
orac = nlp.create_pipe('sentencizer') # 1ª Tuberia/Fase procesamiento. Esto separa la oracion

In [8]:
nlp.add_pipe(orac, before='parser') # Se añade al pipe e identifica 

In [9]:
doc = doc = nlp (text) #Volvemos a pasar el texto

In [10]:
for orac in doc.sents: #Ahora, separa en frase
    print(orac)

Hay un gato en la casa.
Hay una granja en la nevera.
Esta es una tercera frase


In [11]:
# Vamos a eliminar las stop words

stopwords_spacy = list(STOP_WORDS) #Checkar cuales son
print(stopwords_spacy)
len(stopwords_spacy) 

['dice', 'trabajas', 'usais', 'durante', 'mismas', 'propia', 'podria', 'nunca', 'siempre', 'pueda', 'intenta', 'dos', 'cada', 'empleas', 'algunas', 'cosas', 'contigo', 'está', 'qeu', 'pues', 'tus', 'solo', 'aun', 'cuanto', 'usan', 'últimos', 'nuevos', 'nada', 'fuera', 'medio', 'dado', 'también', 'verdadera', 'demasiado', 'ninguna', 'proximo', 'próximo', 'estará', 'en', 'de', 'aseguró', 'pesar', 'ningunas', 'decir', 'próximos', 'nuestro', 'ir', 'lo', 'tengo', 'hay', 'día', 'sobre', 'encuentra', 'buenos', 'tendrán', 'no', 'existen', 'te', 'sabemos', 'pudo', 'ni', 'son', 'es', 'ademas', 'cierta', 'propio', 'otro', 'trata', 'sé', 'poca', 'dar', 'fuimos', 'con', 'última', 'vuestros', 'atras', 'quién', 'breve', 'hoy', 'segun', 'llevar', 'anterior', 'cual', 'poder', 'queremos', 'estais', 'suyas', 'yo', 'aquellas', 'haces', 'mí', 'somos', 'para', 'tuyos', 'total', 'sería', 'estas', 'deben', 'consideró', 'soyos', 'raras', 'todavía', 'buen', 'dentro', 'trabajan', 'explicó', 'nosotras', 'mías', '

551

In [12]:
# Meter?{'todo', 'seáis', 'estada', 'estoy', 'se', 'habrás', 'hubierais', 'hubiésemos', 'esta', 'le', 'habidos', 'estuviesen', 'has', 'suyas', 'ha', 'tú', 'tuviese', 'como', 'eres', 'tenga', 'habíais', 'hubiese', 'sentida', 'fuerais', 'es', 'seré', 'mía', 'tengan', 'me', 'nosotros', 'quienes', 'mi', 'otra', 'estarán', 'hubieseis', 'cuando', 'fueran', 'seamos', 'vosotras', 'o', 'tuya', 'eso', 'fuésemos', 'en', 'siente', 'tendrían', 'hayan', 'el', 'tendrás', 'fui', 'tiene', 'habiendo', 'del', 'habríais', 'estar', 'vuestras', 'nuestro', 'serías', 'estos', 'hayáis', 'seremos', 'tenemos', 'ellas', 'tuvierais', 'teníamos', 'mías', 'una', 'nada', 'habré', 'tu', 'sus', 'hube', 'no', 'estuvieron', 'y', 'estabas', 'fueses', 'sean', 'tendríamos', 'vuestra', 'esto', 'habrán', 'tengo', 'poco', 'tenidas', 'estadas', 'tenida', 'estuve', 'otro', 'míos', 'tuyas', 'al', 'otras', 'muchos', 'estarían', 'estéis', 'sí', 'estuvieras', 'te', 'tuvieran', 'estaremos', 'algo', 'nosotras', 'habéis', 'seríais', 'suyo', 'hubiéramos', 'tuviera', 'habrá', 'estaréis', 'de', 'había', 'tendríais', 'habías', 'hubiesen', 'otros', 'hubiste', 'ti', 'sentido', 'fuimos', 'tuyos', 'porque', 'tengáis', 'qué', 'fuese', 'la', 'su', 'nuestras', 'están', 'tuvimos', 'estuviésemos', 'éramos', 'muy', 'mucho', 'vosotros', 'algunas', 'he', 'sentidos', 'estuviese', 'ella', 'fuiste', 'estarás', 'fuéramos', 'uno', 'tenían', 'fueseis', 'ni', 'suyos', 'estaría', 'nuestros', 'estén', 'os', 'tengamos', 'estarías', 'les', 'estábamos', 'también', 'e', 'tendrán', 'está', 'estés', 'tuviésemos', 'sentidas', 'antes', 'mí', 'estemos', 'estaríais', 'soy', 'estuvisteis', 'quien', 'tuvieses', 'estáis', 'que', 'fueron', 'esas', 'hubimos', 'tuvieras', 'entre', 'tuviste', 'unos', 'esté', 'serán', 'habida', 'estaban', 'hayamos', 'fue', 'todos', 'estaríamos', 'serás', 'ellos', 'estado', 'tenías', 'estando', 'estaba', 'estuviéramos', 'vuestros', 'habrías', 'hemos', 'fuisteis', 'tenidos', 'desde', 'esa', 'habría', 'tenéis', 'estuvierais', 'tendré', 'tendrías', 'hayas', 'hubieron', 'sintiendo', 'con', 'habréis', 'habido', 'tendremos', 'estad', 'teniendo', 'vuestro', 'será', 'ya', 'fuera', 'durante', 'hay', 'hubieses', 'han', 'más', 'estuviera', 'estuvieses', 'habíamos', 'son', 'para', 'tenido', 'tuvieron', 'sois', 'esos', 'por', 'estamos', 'sin', 'ese', 'nos', 'tus', 'tuvieseis', 'cual', 'hubisteis', 'habríamos', 'habrían', 'tenía', 'suya', 'eran', 'contra', 'mis', 'habremos', 'estuviste', 'a', 'hubieras', 'estuvieran', 'hubo', 'sentid', 'sobre', 'seas', 'somos', 'algunos', 'pero', 'eras', 'erais', 'tuve', 'hubiera', 'sería', 'hubieran', 'tened', 'tuyo', 'fuesen', 'tendrá', 'tienen', 'seréis', 'mío', 'sea', 'hasta', 'teníais', 'ante', 'tendría', 'nuestra', 'tuviesen', 'habían', 'habidas', 'este', 'un', 'serían', 'donde', 'lo', 'era', 'fueras', 'haya', 'estados', 'seríamos', 'tienes', 'estará', 'estabais', 'estuvieseis', 'estaré', 'las', 'los', 'tuviéramos', 'estás', 'tuvo', 'tendréis', 'tengas', 'estuvo', 'estas', 'él', 'tanto', 'estuvimos', 'tuvisteis', 'yo'}

In [13]:
for token in doc:
    if token.is_stop == False:
        print(token)

gato
casa
.
granja
nevera
.
frase


# Clasificacion de texto

In [ ]:
#!pip install xmltodict

In [ ]:
#Importar las librerías necesarias para leer los archivos de TASS
import xmltodict
import json
import pandas as pd
import re

In [ ]:
#Traer el archivo xml original xml y convertirlo en un diccionario, escogimos el de España
with open("TASS2019_country_ES_train.xml") as xml_file: # Lo abro
    data_dict = xmltodict.parse(xml_file.read()) # Lo paso a diccionario
xml_file.close()

In [ ]:
#Convertir a json el diccionario
json_data = json.dumps(data_dict)

In [ ]:
#Escribir en un archivo el resultado en json
with open("TASS2019_country_ES_train.json", "w") as json_file:
    json_file.write(json_data)
json_file.close()

In [17]:
#Limpieza de datos en la fuente
#Original en json
fin = open("TASS2019_country_ES_train.json", "rt")
#Archivo resultante en json
fout = open("TASS2019_country_ES_train-sintilde.json", "wt")
#Procesamiento de lìneas del archivo
for line in fin:
	#Reemplazar los caracteres unicode, no se dejaron tildes porque causan error
    strtmp1 = line.replace('\\u00f1', 'ñ')
    strtmp1 = strtmp1.replace('\\u00e1', 'a')
    strtmp1 = strtmp1.replace('\\u00e9', 'e')
    strtmp1 = strtmp1.replace('\\u00ed', 'i')
    strtmp1 = strtmp1.replace('\\u00f3', 'o')
    strtmp1 = strtmp1.replace('\\u00fa', 'u')
    strtmp1 = strtmp1.replace('\\u00bf', '¿')
    strtmp1 = strtmp1.replace('\\u00a1', '¡')
    strtmp1 = strtmp1.replace('\\u00d1', 'Ñ')
    strtmp1 = strtmp1.replace('\\u00c1', 'A')
    strtmp1 = strtmp1.replace('\\u00c9', 'E')
    strtmp1 = strtmp1.replace('\\u00cd', 'I')
    strtmp1 = strtmp1.replace('\\u00d3', 'O')
    strtmp1 = strtmp1.replace('\\u00da', 'U')
    strtmp1 = strtmp1.replace('\\u00fc', 'ü')
    strtmp1 = strtmp1.replace('\\u00b0', '')
    #Quitar el inicio y el fin del json para dejar solo los tweets
    strtmp1 = strtmp1.replace('{"tweets": {"tweet": ', '')
    strtmp1 = strtmp1.replace(']}}', ']')
    #Quitar el diccionario que contiene la polaridad y dejarla solo con su valor de sentimiento
    strtmp1 = strtmp1.replace('"sentiment": {"polarity": {"value": ', '"sentiment": ')
    strtmp1 = strtmp1.replace('"NONE"}}', '"NONE"')
    #Asignamos al sentimiento positivo el valor de 1
    strtmp1 = strtmp1.replace('"P"}}', '1')
    strtmp1 = strtmp1.replace('"NEU"}}', '"NEU"')
    #Asignamos al sentimiento negativo el valor de 0
    strtmp1 = strtmp1.replace('"N"}}', '0')
    #eliminación de puntuaciones
    strtmp1 = re.sub('[¡!#$).;¿?&°]', '', strtmp1.lower())
    fout.write(strtmp1)
#cerrar archivos
fin.close()
fout.close()

In [ ]:
#tomar los datos del archivo creado en un dataframe
train_df = pd.read_json('TASS2019_country_ES_train-sintilde.json')
train_df.head()

#Función para eliminar las menciones a otros usuarios de twitter
def filter_reply(content):
    temp = content
    while temp.find("@") > -1:
        print (temp)
        temp = temp[:temp.find("@")] + temp[(temp.find(" ",temp.find("@"))):]
    return temp

In [ ]:
temp = temp[:temp.find("@")] + temp[(temp.find(" ",temp.find("@"))):]
    
print (temp)

#Quitar menciones del texto
train_df['content'] = train_df['content'].apply(filter_reply)

## Test Final

In [18]:
final_final_resultado = pd.read_csv('final_final_resultado.csv')

for line in final_final_resultado:
	#Reemplazar los caracteres unicode, no se dejaron tildes porque causan error
    strtmp1 = line.replace('\\u00f1', 'ñ')
    strtmp1 = strtmp1.replace('\\u00e1', 'a')
    strtmp1 = strtmp1.replace('\\u00e9', 'e')
    strtmp1 = strtmp1.replace('\\u00ed', 'i')
    strtmp1 = strtmp1.replace('\\u00f3', 'o')
    strtmp1 = strtmp1.replace('\\u00fa', 'u')
    strtmp1 = strtmp1.replace('\\u00bf', '¿')
    strtmp1 = strtmp1.replace('\\u00a1', '¡')
    strtmp1 = strtmp1.replace('\\u00d1', 'Ñ')
    strtmp1 = strtmp1.replace('\\u00c1', 'A')
    strtmp1 = strtmp1.replace('\\u00c9', 'E')
    strtmp1 = strtmp1.replace('\\u00cd', 'I')
    strtmp1 = strtmp1.replace('\\u00d3', 'O')
    strtmp1 = strtmp1.replace('\\u00da', 'U')
    strtmp1 = strtmp1.replace('\\u00fc', 'ü')
    strtmp1 = strtmp1.replace('\\u00b0', '')
    #Quitar el inicio y el fin del json para dejar solo los tweets
    strtmp1 = strtmp1.replace('{"tweets": {"tweet": ', '')
    strtmp1 = strtmp1.replace(']}}', ']')
    #Quitar el diccionario que contiene la polaridad y dejarla solo con su valor de sentimiento
    strtmp1 = strtmp1.replace('"sentiment": {"polarity": {"value": ', '"sentiment": ')
    strtmp1 = strtmp1.replace('"NONE"}}', '"NONE"')
    #Asignamos al sentimiento positivo el valor de 1
    strtmp1 = strtmp1.replace('"P"}}', '1')
    strtmp1 = strtmp1.replace('"NEU"}}', '"NEU"')
    #Asignamos al sentimiento negativo el valor de 0
    strtmp1 = strtmp1.replace('"N"}}', '0')
    #eliminación de puntuaciones
    strtmp1 = re.sub('[¡!#$).;¿?&°]', '', strtmp1.lower())


In [19]:
#Quitar columnas sin clasificación de sentimiento 
indexNames = final_final_final_resultado[(final_final_final_resultado['sentiment'] == 'none') | (final_final_final_resultado['sentiment'] == 'neu')].index
final_final_final_resultado.drop(indexNames , inplace=True)
final_final_final_resultado.head()

,tweetid,user,content,date,lang,sentiment
0,768213567418036224,3429794128,@myendlesshazza a que puto mal escribo b me si...,2016-08-23 22:29:21+00:00,es,0
1,768221670255493120,396616007,quiero mogollon a @albabenito99 pero sobretodo...,2016-08-23 23:01:33+00:00,es,1
2,768221021300264960,2845050061,vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58+00:00,es,0
3,768220253730009088,442100979,@yulian_poe @guillermoterry1 ah mucho mas por ...,2016-08-23 22:55:55+00:00,es,1
4,768231229439311872,529648312,@toni_end seria mejor que dejasen de emitir es...,2016-08-23 23:39:32+00:00,es,0


In [20]:
#Importar librerías de aprendizaje
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [21]:
#Verificar frecuencias de cada categoría
final_final_final_resultado['sentiment'].value_counts()

0    2854
1    1896
Name: sentiment, dtype: int64

In [22]:
#Verificar si hay datos nulos
final_final_final_resultado.isnull().sum()

tweetid      0
user         0
content      0
date         0
lang         0
sentiment    0
dtype: int64

# Tokenización

In [23]:
import string #Quita todos los simbolos
puntua = string.punctuation + "¡¿"
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿'

In [24]:
#Función para limpieza de datos
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
    
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
    
    return clean_tokens

In [25]:
text_data_cleaning("¡Hola cómo estás!. ¿Te gusta el meetup?")

['Hola', 'Te', 'gustar', 'meetup']

# Vectorization Feature Engineering (TF-IDF)

In [26]:
#importar librería de vectorización
from sklearn.svm import LinearSVC

In [27]:
#Definir la función de tokenizado y crear el clasificador lineal
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [28]:
#Crear los vectores de datos
X = final_final_final_resultado['content']
y = final_final_final_resultado['sentiment']

In [29]:
#Crear el vector de entrenamiento como una porción de los datos y dejar el resto para pruebas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3800,), (950,), (3800,), (950,))

In [30]:
#Crear un pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [31]:
#evitar que el formato se tome como unknown
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [32]:
#Entrenar el clasificador
clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f8e47f58d30>)),
                ('clf', LinearSVC())])

In [33]:
#Crear el vectos de valores predichos a partir del clasificador
y_pred = clf.predict(X_test)

In [34]:
#Ver la precisión obtenida
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       578
           1       0.84      0.78      0.81       372

    accuracy                           0.86       950
   macro avg       0.85      0.84      0.85       950
weighted avg       0.86      0.86      0.85       950



In [35]:
# 86% de 'efectividad'

In [36]:
#Crear la matriz de confusión
confusion_matrix(y_test, y_pred)

array([[522,  56],
       [ 81, 291]])

Me dan 82 falsos negativos

In [45]:
#Predecir algunas frases de prueba
clf.predict([''])

array([0])

# Función-Modelo

In [49]:
import tweepy

In [50]:
# 4 authentication chains

consumer_key='jQTzrkE7vlZbg2ntJu4LESCZs'
consumer_secret='AS4B8YLOWXMcrHjJyZ8stWcm9Cp2qh0rCdIjiWaPBaTTc22tnO'
access_key='902474996-b1ltSFx5Y2EdJfi2s63pghsULdjLTF1lkW6oHBvj'
access_secret='TrZr5nGmi2Q4RcAJJ3UHbwuWkAqjcNXOTfCHLm1eOzpnn'

# authorize twitter, initialize tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [56]:
bully = pd.DataFrame()

In [59]:
mentions = api.mentions_timeline(tweet_mode= 'extented')
    
user = []

Bully = pd.DataFrame()
    
for tweet in mentions:
        
    if clf.predict([tweet.text]) == 0:
        print('OKKKKKKKKK')
        
        api.create_block(tweet.user.screen_name)
        
        
    elif clf.predict([tweet.text]) == 1:
        print('BUUUUENO')
        

    #print(tweet.created_at)
    #print(tweet.text)
    #print(tweet.source)
    #print(tweet.user.name)
        
    #user.append(tweet.User)
        
#for i in tweet.user:
        
        #name = i.name
        #screen_name = i.screen_name
        #location = i.location if i.location else ''
        #url = 'https://twitter.com/' + i.screen_name
        #followers_count = i.followers_count
        #verified = i.verified

OKKKKKKKKK


TweepError: Read-only application cannot POST.

In [ ]:
# Checkear menciones

def check_mentions():
    
    mentions = api.mentions_timeline(tweet_mode= 'extented')
    
    user = []
    
    for tweet in mentions:
        
        
        print(tweet.created_at)
        print(tweet.text)
        print(tweet.source)
        print(tweet.user.name)
        
        #user.append(tweet.User)
        
    #for i in tweet.user:
        
            #name = i.name
            #screen_name = i.screen_name
            #location = i.location if i.location else ''
            #url = 'https://twitter.com/' + i.screen_name
            #followers_count = i.followers_count
            #verified = i.verified

In [ ]:
check_mentions()